# Pre-training a Small Language Model (SLM)

## Imports

In [ ]:
import re
from tokenizer_slm import SimpleTokenizer
import os
import sys

os.environ["KERAS_BACKEND"] = "jax" # Set a parameter for Keras.

In [3]:
%%capture
!pip install "git+https://github.com/google-deepmind/ai-foundations.git@main"

import keras
import pandas as pd
import tensorflow as tf

In [4]:
from ai_foundations import training
from ai_foundations import generation
from ai_foundations import visualizations

keras.utils.set_random_seed(812)  # For Keras layers.

## Load and tokenize dataset

Tokenize the dataset and map tokens to token ID for preprocessing

In [5]:
from typing import List, Dict

class SimpleTokenizer:

    UNKNOWN_TOKEN = "<UNK>"
    PAD_TOKEN = "<PAD>"

    def __init__(self, corpus: List[str], vocabulary: List[str] | None = None):
        """Initializes the tokenizer with texts in the dataset or with a vocabulary."""

        if vocabulary is None:
            if isinstance(corpus, str):
                corpus = [corpus]

            # Convert text sequence to tokens.
            tokens = []
            for text in corpus:
                for token in self.space_tokenizer(text):
                    tokens.append(token)

            # Create a vocabulary comprising of unique tokens.
            self.vocabulary = self.build_vocabulary(tokens)

        else:
            self.vocabulary = vocabulary

        self.vocabulary = [self.PAD_TOKEN] + self.vocabulary + [self.UNKNOWN_TOKEN]
        self.vocabulary_size = len(self.vocabulary)

        # Create token and token IDs mappings.
        self.token_to_index = {}
        self.index_to_token = {}

        for index, token in enumerate(self.vocabulary):
            self.token_to_index[token] = index
            self.index_to_token[index] = token

        self.unk_index = self.token_to_index[self.UNKNOWN_TOKEN]
        self.pad_index = self.token_to_index[self.PAD_TOKEN]

    def space_tokenizer(self, text: str) -> List[str]:
        """Splits a string into a list of tokens and removes punctuation."""

        # Replace exclamation marks with a space before splitting
        text = re.sub(r'[!;:()"\[\]{}<>/\\`~@#$%^&*\_+=|\n“”]', ' ', text)
        tokens = re.split(r'\s+', text)
        tokens = [token for token in tokens if token]
        return tokens

    def build_vocabulary(self, tokens: List[str]) -> List[str]:
        """Builds a vocabulary from a list of tokens."""
        return list(set(tokens))

    def encode(self, text: str) -> List[int]:
        """Encodes a text sequence into a list of indices."""

        # Convert tokens into indices.
        indices = []
        for token in self.space_tokenizer(text):
            token_index = self.token_to_index.get(token, self.unk_index)
            indices.append(token_index)

        return indices

    def decode(self, indices: int | List[int]) -> str:
        """Decodes a list (or single index) of integers back into tokens."""

        # If a single integer is passed, convert it into a list.
        if isinstance(indices, int):
            indices = [indices]

        # Map indices to tokens.
        tokens = []
        for index in indices:
            token = self.index_to_token.get(index, self.unk_index)
            tokens.append(token)

        return " ".join(tokens)


In [6]:
from datasets import load_dataset

naija_web = load_dataset("saheedniyi/naijaweb")
dataset = naija_web["train"]["text"][:500]


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00003.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

data/train-00001-of-00003.parquet:   0%|          | 0.00/218M [00:00<?, ?B/s]

data/train-00002-of-00003.parquet:   0%|          | 0.00/223M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/270137 [00:00<?, ? examples/s]

In [7]:
tokenizer = SimpleTokenizer(dataset)

In [8]:
encoded_tokens = []

for text in dataset:
  token_id = tokenizer.encode(text)
  encoded_tokens.append(token_id)

In [9]:
encoded_tokens[0][:20]

[13433,
 12532,
 16272,
 13185,
 32214,
 8451,
 26466,
 9452,
 10954,
 13433,
 12532,
 16272,
 13185,
 32214,
 31034,
 24250,
 14738,
 29798,
 12284,
 15387]

## Pad the dataset

The training data will be inputted as a matrix of the token IDs. In order to properly process, the matrix rows have to be of equal size. Hence, we need to pad smaller row tokens with the pad tokens `<PAD>`, and truncate longer row tokens for uniform length

In [10]:
shortest_paragraph_length = len(min(encoded_tokens, key=len))
longest_paragraph_length = len(max(encoded_tokens, key=len))

print(f"Shortest paragraph:", shortest_paragraph_length)
print(f"Longest paragraph is:", longest_paragraph_length)

Shortest paragraph: 59
Longest paragraph is: 4806


In [11]:
max_length = 1000 # Deu to GPU limits, set to 1000

padded_sequences = keras.preprocessing.sequence.pad_sequences(
    encoded_tokens,
    maxlen=max_length,
    padding="post",
    truncating="post",
    value=tokenizer.pad_index,
)

print("New length of first paragraph:", len(padded_sequences[0]), "\n")

print("Padded tokens of first paragraph:\n", padded_sequences[0])

New length of first paragraph: 1000 

Padded tokens of first paragraph:
 [13433 12532 16272 13185 32214  8451 26466  9452 10954 13433 12532 16272
 13185 32214 31034 24250 14738 29798 12284 15387  3978 12535  2344 10685
 18653 23055 10685 12992 13554 32343  6394 30032 17224 32870 33582 17177
 10765 13433  1228 12417 33582 29398  5631 32870 15074  1296 33582   980
 22877 23741   457 33582  7384 13185 14091 12778 26872 32682 34113  1431
 11917 32214 21829 26757 12416 26757 32870 29012 30063 13185 33582 17177
 21839 10685 20948 33582 22877 23741 29328 10290 33582 21829  5939  3978
 19375 17032 10685 22925 10685  8654  4542  7460  2794 27084 10685 27149
 27760 29798  3978 12535  5526 22877 15355 20871  6394 30032 17287  8178
  2794 33341 34084 32985   457 33582 25025 21693 13185 14091 10683 26872
 32682 27126  1431 11917 21099 12742  6198 33582 13433 22923   819 32009
 27634 29798 15387 29572  7020 33582 24406 24022 26346 26465 10685 33582
  2771  8924 26872 29012  6064 23174  3716 11917 33

## Prepare input and target

In [12]:
# Extract all tokens except the last one.
input_sequences = padded_sequences[:, :-1]

# Extract all tokens except the first one.
target_sequences = padded_sequences[:, 1:]

In [14]:
max_length = input_sequences.shape[1]

## Shuffle the dataset and set the batch size

Now, we have to shuffle the dataset and break them down into batches. Shuffling is so the model doesn't overfit to the current order of the dataset and batches make it easier to process



In [15]:
# Create TensorFlow dataset to prepare sequences.
tf_dataset = tf.data.Dataset.from_tensor_slices((input_sequences, target_sequences))

# Randomly shuffle the dataset.
tf_dataset = tf_dataset.shuffle(buffer_size=len(input_sequences))

batch_size = 2

# Create batches.
batches = tf_dataset.batch(batch_size)

for batch in batches.take(1):
    print(batch)

(<tf.Tensor: shape=(2, 999), dtype=int32, numpy=
array([[10765,  2477, 27753, ...,     0,     0,     0],
       [29798, 12284, 15387, ..., 21468, 26872, 33582]], dtype=int32)>, <tf.Tensor: shape=(2, 999), dtype=int32, numpy=
array([[ 2477, 27753, 32945, ...,     0,     0,     0],
       [12284, 15387, 10818, ..., 26872, 33582,  2477]], dtype=int32)>)


In [16]:
total_batches = 0
for batch in batches:
    total_batches += 1
print("Total number of batches is:", total_batches)

Total number of batches is: 250


## Train the language model

### Initialize the model

This is a module provided by Google Deepmind that sets up a 3.5 million parameters language model for pre-training.

In [17]:
model = training.create_model(
    vocabulary_size=tokenizer.vocabulary_size,
    max_length=max_length,
    learning_rate=1e-4
)

### Initialize a callback function

This function can be used to easily test the model's next token prediction over training iterations or epoch

In [18]:
prompt = "Governor"
prompt_ids = tokenizer.encode(prompt)
callback_func = training.TextGenerator(
    max_tokens=10, start_tokens=prompt_ids, tokenizer=tokenizer, print_every=10
)

### Train the model

In [21]:
num_epochs = 200

history = model.fit(
    x=batches, verbose=2, epochs=num_epochs, callbacks=[callback_func]
)

Epoch 1/200
250/250 - 12s - 48ms/step - loss: 5.5380
Epoch 2/200
250/250 - 12s - 49ms/step - loss: 5.3665
Epoch 3/200
250/250 - 13s - 51ms/step - loss: 5.1929
Epoch 4/200
250/250 - 13s - 54ms/step - loss: 5.0356
Epoch 5/200
250/250 - 14s - 54ms/step - loss: 4.8665
Epoch 6/200
250/250 - 13s - 52ms/step - loss: 4.7133
Epoch 7/200
250/250 - 13s - 51ms/step - loss: 4.5609
Epoch 8/200
250/250 - 13s - 51ms/step - loss: 4.4083
Epoch 9/200
250/250 - 13s - 51ms/step - loss: 4.2536
Epoch 10/200
Generated text:
 Governor Wike the spirit of Lagos State. Former death Read Also 

250/250 - 26s - 104ms/step - loss: 4.1034
Epoch 11/200
250/250 - 13s - 52ms/step - loss: 3.9614
Epoch 12/200
250/250 - 14s - 54ms/step - loss: 3.8169
Epoch 13/200
250/250 - 14s - 54ms/step - loss: 3.6844
Epoch 14/200
250/250 - 13s - 52ms/step - loss: 3.5470
Epoch 15/200
250/250 - 13s - 51ms/step - loss: 3.4126
Epoch 16/200
250/250 - 13s - 51ms/step - loss: 3.2963
Epoch 17/200
250/250 - 13s - 52ms/step - loss: 3.1796
Epoch 1

## Evaluate model

In [28]:
prompt = "Why is the"
num_tokens_to_generate = 10
generated_text, probs = generation.generate_text(
    prompt,
    num_tokens_to_generate,
    model=model,
    tokenizer=tokenizer,
    pad_token_id=tokenizer.pad_index,
    sampling_mode="greedy" # To generate the highest probability generation.
)

print("Generated text:", generated_text, "\n")
print("Probabilities of each token:", probs)
visualizations.plot_next_token(probs[0], prompt=prompt, tokenizer=tokenizer)

Generated text: Why is the mighty fallen? The good news is that some Nigerians are 

Probabilities of each token: [Array([2.9727426e-10, 2.1516307e-09, 7.6290780e-06, ..., 3.8094616e-10,
       8.8308603e-11, 3.0756722e-10], dtype=float32), Array([1.8351048e-11, 1.2976673e-12, 3.9642361e-12, ..., 3.2015777e-17,
       7.8059507e-14, 2.2314222e-11], dtype=float32), Array([1.3000180e-16, 2.5452838e-15, 3.3471549e-15, ..., 9.0895717e-20,
       2.6021544e-17, 1.2084170e-16], dtype=float32), Array([3.0091307e-15, 6.9192848e-14, 3.8776826e-10, ..., 4.3637492e-13,
       3.6422154e-15, 3.3498154e-15], dtype=float32), Array([2.6435965e-12, 2.0224024e-11, 1.2084537e-08, ..., 5.4529919e-15,
       3.4398252e-13, 3.0816536e-12], dtype=float32), Array([2.3871397e-17, 4.2750458e-13, 3.8596333e-17, ..., 2.2288818e-13,
       7.1858384e-17, 2.6987788e-17], dtype=float32), Array([2.10756604e-18, 1.38611995e-14, 1.59900607e-10, ...,
       8.25776814e-16, 1.86310968e-17, 2.80918622e-18], dtype=float32